In [1]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [16]:
import matplotlib.pyplot as plt
import numpy as np

from gspx.utils.graph import make_sensor
from gspx.utils.display import plot_graph
from gspx.signals import QuaternionSignal, Quaternion
from gspx.qgsp import QMatrix
from gspx.adaptive import QLMS
from gspx.utils.quaternion_matrix import explode_quaternions
from gspx.datasets import WeatherGraphData, uk_weather

In [21]:
uk_data = WeatherGraphData()
A, coords = uk_data.graph
s = uk_data.signal

In [25]:
A = QMatrix([A, A, A, A])

ValueError: could not broadcast input array from shape (1,31329) into shape (4,1)

In [20]:
A.matrix == Quaternion(0.0, 0.0, 0.0, 0.0)

array([[ True, False, False, False, False, False],
       [False,  True, False, False, False, False],
       [False, False,  True, False, False, False],
       [False, False, False,  True, False, False],
       [False, False, False, False,  True, False],
       [False, False, False, False, False,  True]])

<177x177 sparse matrix of type '<class 'numpy.float64'>'
	with 1593 stored elements in Compressed Sparse Row format>

In [ ]:
N = 6
A1, coords = make_sensor(N, seed=2)
Ai, _ = make_sensor(N, seed=3)
Aj, _ = make_sensor(N, seed=4)
Ak, _ = make_sensor(N, seed=5)

X = QMatrix([A1, Ai, Aj, Ak])

In [ ]:
self = QLMS()

m, d = X.shape
X_ = QLMS.add_intercept_term(X)
X_, mu, std = QLMS.normal_scaling(X_, ignore_first=True)

self.mu_ = mu
self.std_ = std

In [ ]:
theta = QLMS.initiate(length=d + 1, method='zeros')
J = np.zeros(self.max_iter)

err = X_ * theta - y

In [ ]:
res = {}
for lr in self.alpha:
    theta = QLMS.initiate(length=d + 1, method='zeros')
    J = np.zeros(self.max_iter)

    for idx_iter in range(self.max_iter):
        err = X_ @ theta - y

        # Calculate the J term, which is the current MSE
        J[idx_iter] = (0.5/m) * err.T @ err

        # The gradient
        grad = (1/m) * np.conjugate(X_).T @ err

        # Here is the actual update
        theta = theta - lr * grad
    res[lr] = dict(result=theta, cost=J)

self.res_ = res
idx_lower_cost = np.argmin([v['cost'][-1] for k, v in res.items()])
self.best_lr_ = self.alpha[idx_lower_cost]

return self